# Create Network Graph


Basic setup


In [5]:
import pandas as pd
import numpy as np
from pyvis.network import Network
import networkx as nx
import seaborn as sns

Loading previously calculated dataframes


In [6]:
dataframe_dir = "MediumArticles"
df = pd.read_csv(f"./data_output/{dataframe_dir}/chunks.csv", sep="|")
df_concepts = pd.read_csv(f"./data_output/{dataframe_dir}/concepts.csv", sep="|")
print(df_concepts.shape)
df_concepts.head()

FileNotFoundError: [Errno 2] No such file or directory: './data_output/MediumArticles/chunks.csv'

## Calculate Graph Dataframe


Graph dataframe is a dataframe where every row is a connection between two nodes.

It is basically an inner self join of the nodes dataframe


In [240]:
dfne_join = pd.merge(
    df_concepts, df_concepts, how="inner", on="chunk_id", suffixes=("_L", "_R")
)

## Remove self Loops
self_loops_drop = dfne_join[dfne_join["entity_L"] == dfne_join["entity_R"]].index
dfg = dfne_join.drop(index=self_loops_drop).reset_index()

## This is our graph dataframe
print("Total number of nodes = ", dfg.shape[0])
dfg.head()

Total number of nodes =  2570


,index,entity_L,importance_L,category_L,chunk_id,type_L,entity_R,importance_R,category_R,type_R
0,1,interoperability,5,concept,bb894d4a987241c6b7827d4acabac6dc,concept,introduction,3,event,concept
1,2,introduction,3,event,bb894d4a987241c6b7827d4acabac6dc,concept,interoperability,5,concept,concept
2,5,unique charging port design,4,misc,ebcfb9727a42482d986fa2b0a64fa8ac,concept,usb c,5,object,concept
3,6,unique charging port design,4,misc,ebcfb9727a42482d986fa2b0a64fa8ac,concept,mobile phone model,3,object,concept
4,7,unique charging port design,4,misc,ebcfb9727a42482d986fa2b0a64fa8ac,concept,phone companies,4,organisation,concept


#### Clean the graph dataframe


The original graph dataframe is too big to visualise. So we will make another dataframe for visualisation purpose.

-   remove the less important nodes
-   remove less important edges


In [259]:
## Drop nodes which are less important
less_important_nodes = dfg[(dfg["importance_L"] < 2)].index
## Drop edges where both the nodes are less important than 5
less_important_edges = dfg[(dfg["importance_L"] < 2) & (dfg["importance_R"] < 2)].index
drops = less_important_nodes.union(less_important_edges)

print(
    "Less important Nodes = ",
    less_important_nodes.shape[0],
    "\nLess Important Edges = ",
    less_important_edges.shape[0],
)

## Remove these rows from the graph dataframe
dfg_vis = dfg.drop(index=drops).reset_index()


Less important Nodes =  29 
Less Important Edges =  2


### Combine similar edges

Group the edges between the same nodes and combine them into single edge with its weight equal to the count. 

In [260]:

## Group and aggregate edges.
dfg_vis = (
    dfg_vis.groupby(["entity_L", "entity_R"])
    .agg(
        {
            "importance_L": "mean",
            "importance_R": "mean",
            "chunk_id": [",".join, "count"],
        }
    )
    .reset_index()
)
dfg_vis.columns = [
    "entity_L",
    "entity_R",
    "importance_L",
    "importance_R",
    "chunks",
    "count",
]

print("Final Number of Edges in the Visualisation Graph = ", dfg_vis.shape[0])
dfg_vis.head()

Final Number of Edges in the Visualisation Graph =  2481


,entity_L,entity_R,importance_L,importance_R,chunks,count
0,10000 w 100th ave,35200,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
1,10000 w 100th ave,35209,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
2,10000 w 100th ave,99def,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
3,10000 w 100th ave,adt_a01,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
4,10000 w 100th ave,al,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1


### Removing overconnected nodes

These are featured in the header and the footer of the pdf file, so they are a little too connected.


In [261]:
ind = dfg_vis[
    dfg_vis["entity_L"].isin(
        ["Pathways to Health Equity for the G20", "Accelerating Global Health"]
    )
    | dfg_vis["entity_R"].isin(
        ["Pathways to Health Equity for the G20", "Accelerating Global Health"]
    )
].index
dfg_vis.drop(index=ind, axis=1, inplace=True)
print("Final Number of Edges  = ", dfg_vis.shape[0], "\nDropped edges:", len(ind))
dfg_vis.head()

Final Number of Edges  =  2481 
Dropped edges: 0


,entity_L,entity_R,importance_L,importance_R,chunks,count
0,10000 w 100th ave,35200,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
1,10000 w 100th ave,35209,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
2,10000 w 100th ave,99def,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
3,10000 w 100th ave,adt_a01,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1
4,10000 w 100th ave,al,2.0,2.0,45a6201014f84cc7bcb7aea24a7f7af6,1


## Creating a NetworkX Graph


Calculate nodes

Here I am grouping the graph dataframe by left node and calculating the mean importance. This way we will end up with only the unique nodes from the graph dataframe along with their weights.


In [262]:
# nodes = df_graph["entity_L"].unique()
nodes = dfg_vis.groupby(["entity_L"]).agg({"importance_L": "mean"}).reset_index()
nodes.head()

,entity_L,importance_L
0,10000 w 100th ave,2.0
1,2010,4.0
2,30-year-old standard,4.0
3,35200,2.0
4,35209,2.0


Build a NetworkX object with nodes and edges


In [263]:
G = nx.Graph()
for index, row in nodes.iterrows():
    G.add_node(row["entity_L"])

for index, row in dfg_vis.iterrows():
    G.add_edge(str(row["entity_L"]), str(row["entity_R"]))

### Community Detection


Detect communities using the Girvan Newman algorithm


In [264]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))

Number of Communities =  27


Add colors to nodes based on community


In [265]:
palette = "hls"


## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"entity_L": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)

df_nodes_colors = pd.merge(
    nodes, colors, how="left", on="entity_L", suffixes=("_N", "_C")
)
# nodes.head()
df_nodes_colors.head()

,entity_L,importance_L,color,group
0,10000 w 100th ave,2.0,#db576c,1
1,2010,4.0,#db578a,2
2,30-year-old standard,4.0,#db57a7,3
3,35200,2.0,#db576c,1
4,35209,2.0,#db576c,1


So now we have a nodes dataframe with colors and sizes of each node.

lets recreate our graph.


In [266]:
G = nx.Graph()
node_size_multiple = 6

for index, row in df_nodes_colors.iterrows():
    G.add_node(
        row["entity_L"],
        size=row["importance_L"] * node_size_multiple,
        title=row["entity_L"],
        color=row["color"],
    )

for index, row in dfg_vis.iterrows():
    G.add_edge(
        str(row["entity_L"]),
        str(row["entity_R"]),
        weight=row["count"],
        name=row["chunks"],
    )

## Visualisation


In [267]:
graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    font_color="#cccccc",
    # filter_menu=True,
)

net.from_nx(G)
net.repulsion(node_distance=150, spring_length=400)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
